<a href="https://colab.research.google.com/github/GauravPatel89/EVA-Track3-Assignments/blob/master/endGame/endGameTD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twin-Delayed DDPG


In [0]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Work in endGameTD3 in google drive so that even after session ends work will be available
%cd '/content/drive/My Drive/endGameTD3'
!pwd

/content/drive/My Drive/endGameTD3
/content/drive/My Drive/endGameTD3


To run rest of the code following files should be available in working directory
1. car.png
2. citymap.png
3. MASK1.png
4. endGameEnv.py
5. endGameModels.py
6. endGameUtilities.py

These files can be found [here.](https://github.com/GauravPatel89/EVA-Track3-Assignments/tree/master/EndGame)

## Importing the libraries

In [0]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque

from endGameEnv import car,carEndgameEnv
from endGameModels import ReplayBuffer,Actor,Critic,TD3

In [0]:
def evaluate_policy(policy, eval_episodes=3):
    avg_reward = 0.
    print('')
    print("----------- Evaluation -------------")
    for _ in range(eval_episodes):
        obs = env.reset()
        print(env.car)
        done = False
        while not done:
            action = policy.select_action(np.array(obs))
            obs, reward, done = env.step(action)
            avg_reward += reward
        #print('episode done')
    avg_reward /= eval_episodes
    print ("---------------------------------------")
    print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
    print ("---------------------------------------")
    print('')
    return avg_reward


## We set the parameters

In [0]:
env_name = "carEndGameEnv" 
seed = 0 
save_models = True 

## We create a file name for the two saved models: the Actor and Critic models

In [0]:
file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Settings: TD3_carEndGameEnv_0
---------------------------------------


## We create a folder inside which will be saved the trained models

In [0]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

## We create the carEndGame environment

In [0]:
car1 = car(carImgPath='./car.png',size= (20,10),velocity=(2.0,0.0),angle=0.0)
env = carEndgameEnv(mapImgPath='./citymap.png',maskImgPath='./MASK1.png',carObj=car1)

## We set seeds and we get the necessary information on the states and actions in the chosen environment

In [0]:
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.state_dim
action_dim = env.action_dim
max_action = env.max_action

## We create the policy network

In [0]:
policy = TD3(state_dim, action_dim, max_action)

## We create the Experience Replay memory

In [0]:
replay_buffer = ReplayBuffer()

## We define a list where all the evaluation results over 3 episodes are stored

In [0]:
evaluations = [evaluate_policy(policy)]


----------- Evaluation -------------
imgPath:./car.png, width:20, height:10, velocityX:2.0, velocityY:0,angle:[10.], posX:188, posY:457
Episode Done:Car stuck
Episode steps: 1290  Sand count:  1130  Road Count:  160
imgPath:./car.png, width:20, height:10, velocityX:2.0, velocityY:0,angle:[10.], posX:188, posY:457
Episode Done:Car stuck
Episode steps: 1290  Sand count:  1130  Road Count:  160
imgPath:./car.png, width:20, height:10, velocityX:2.0, velocityY:0,angle:[10.], posX:188, posY:457
Episode Done:Car stuck
Episode steps: 1095  Sand count:  915  Road Count:  180
---------------------------------------
Average Reward over the Evaluation Step: -5791.133333
---------------------------------------



## We initialize the variables

In [0]:
start_timesteps = 5e3  # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 2e3  # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 2e5 # Total number of iterations/timesteps
expl_noise = 0.1  # Exploration noise - STD value of exploration Gaussian noise
batch_size = 100  # Size of the batch
discount = 0.99  # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.005  # Target network update rate
policy_noise = 0.2  # STD of Gaussian noise added to the actions for the exploration purposes
noise_clip = 0.5  # Maximum value of the Gaussian noise added to the actions (policy)
policy_freq = 2  # Number of iterations to wait before the policy network (Actor model) is updated

In [0]:
# Initialize training specific variables
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()
episode_reward = 0
episode_timesteps = 0

## Training

In [0]:
max_timesteps = 300000
while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:

    # If we are not at the very beginning, we start the training process of the model
    if total_timesteps != 0:
      print("Total Timesteps: {} Episode Num: {} Reward: {}".format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    # We evaluate the episode and we save the policy
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      fName = '%s_%d'%(file_name,total_timesteps)
      policy.save(fName, directory="./pytorch_models")  # save current model with current timestep as name subscript
      np.save("./results/%s_%d" % (file_name,total_timesteps), evaluations)
    
    # When the training step is done, we reset the state of the environment
    obs = env.reset()
    print(env.car)
    
    # Set the Done to False
    done = False
    
    # Set rewards and episode timesteps to zero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Before 10000 timesteps, we play random actions
  if total_timesteps < start_timesteps:
    action = env.sampleActionSpace()
  else: # After 10000 timesteps, we switch to the model
    action = policy.select_action(np.array(obs))
    # If the explore_noise parameter is not 0, we add noise to the action and we clip it
    if expl_noise != 0:
      action = (action + np.random.normal(0, expl_noise, size=env.action_dim)).clip(-1*env.max_action, env.max_action)
  
  # The agent performs the action in the environment, then reaches the next state and receives the reward
  new_obs, reward, done = env.step(action)
  
  # We check if the episode is done
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  
  # We increase the total reward
  episode_reward += reward
  
  # We store the new transition into the Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, new_obs, action, reward, done_bool))

  # We update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1 

# We add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s_%d" % (file_name,total_timesteps), evaluations)

imgPath:./car.png, width:20, height:10, velocityX:2.0, velocityY:0,angle:[10.], posX:188, posY:457
Episode Done:Car stuck
Episode steps: 1093  Sand count:  903  Road Count:  190
Total Timesteps: 1093 Episode Num: 1 Reward: -12112.199999999993
Avg CriticLoss:  50.29796984429111  Avg ActorLoss  7.875348937353468  ActorLR:  [0.0001]  CriticLR:  [0.0001]
imgPath:./car.png, width:20, height:10, velocityX:2.0, velocityY:0,angle:[10.], posX:188, posY:457
Episode Done:Car stuck
Episode steps: 1026  Sand count:  845  Road Count:  181
Total Timesteps: 2119 Episode Num: 2 Reward: -4938.100000000008
Avg CriticLoss:  35.47552309166386  Avg ActorLoss  16.411906549107957  ActorLR:  [0.0001]  CriticLR:  [0.0001]

----------- Evaluation -------------
imgPath:./car.png, width:20, height:10, velocityX:2.0, velocityY:0,angle:[10.], posX:188, posY:457
Goal:( 260 476 )
Episode Done:Max episode steps exhausted
Episode steps: 5000  Sand count:  3148  Road Count:  1852
imgPath:./car.png, width:20, height:10, v